In [1]:
import os
import sys

notebook_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

from Classification import classification_class as classification
from Features_extraction import feature_extraction_class as feature_extraction
from Data_processing.images_processing_class import ImagesProcessing
from utils import perform_classification, best_preprocessing

import pandas as pd
import numpy as np
import cv2
import json

In [2]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }

name_best_models = {}

In [3]:
data_normal_path = "../Data/normal/"
data_pothole_path = "../Data/potholes/"

image_process = ImagesProcessing(folder_normal=data_normal_path, folder_potholes=data_pothole_path, img_size=(256, 256))
images = image_process.images
Y = image_process.labels
df_Y = pd.DataFrame(Y)

In [4]:
env_feature_extraction_unprocessed_images = feature_extraction.FeatureExtraction(image_process)

In [5]:
methods = {
    "SIFT": env_feature_extraction_unprocessed_images.method_SIFT,
    "ORB": env_feature_extraction_unprocessed_images.method_ORB,
    "Harris": env_feature_extraction_unprocessed_images.method_Harris,
    "EDGE": env_feature_extraction_unprocessed_images.method_EDGE,
    "Otsu": env_feature_extraction_unprocessed_images.method_otsu,
    "Adaptive": env_feature_extraction_unprocessed_images.method_adaptive,
    "Gabor": env_feature_extraction_unprocessed_images.method_Gabor,
    "LBP": env_feature_extraction_unprocessed_images.method_LBP,
    #"HOG": env_feature_extraction_unprocessed_images.method_HOG
}

for method_name, method_function in methods.items():

    metrics_results_best_methods = perform_classification(
        method_name=method_name, 
        df_Y=df_Y,
        name_best_models=name_best_models,
        metrics_results_best_methods=metrics_results_best_methods,
        feature_extraction_method=method_function 
    )


Extracting SIFT Features
Performing Classification for SIFT
Best method name for SIFT: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.84      0.85        86
           1       0.74      0.78      0.76        50

    accuracy                           0.82       136
   macro avg       0.80      0.81      0.80       136
weighted avg       0.82      0.82      0.82       136

Extracting ORB Features
Performing Classification for ORB
Best method name for ORB: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.63      0.68        86
           1       0.48      0.60      0.54        50

    accuracy                           0.62       136
   macro avg       0.61      0.61      0.61       136
weighted avg       0.64      0.62      0.62       136

Extracting Harris Corner Features
Performing Classification for Harris
Best method name for Harris: RandomForest
Cl

Find best configurations for preprocessing

In [7]:
# TODO: Change this filter with the good association method-filter find in articles



filters = {
    "SIFT": ["bilateral"],  
    "ORB": ["bilateral"],
    "Harris": ["bilateral"],
    "EDGE": ["bilateral"],
    "LBP": ["gaussian"],
    "Gabor": ["bilateral"],
    "otsu": ["gaussian"],
    "adaptive": ["bilateral"]
}

best_configs = {}

for method_name, filter_names in filters.items():
    if isinstance(filter_names, list):  
        best_configs[method_name] = []
        for filter_name in filter_names:
            best_config, all_results = best_preprocessing(
                image_process,
                getattr(env_feature_extraction_unprocessed_images, f"method_{method_name}"),
                filter_name,
                method_name,
                n_iter=3,
                fixed_histogram_method="clahe" if method_name == "SIFT" else None  # ✅ On force CLAHE ici !
            )
            best_configs[method_name].append(best_config)
    else:  
        best_config, all_results = best_preprocessing(
            image_process,
            getattr(env_feature_extraction_unprocessed_images, f"method_{method_name}"),
            filter_names,
            method_name,
            n_iter=3
        )
        best_configs[method_name] = best_config



Extracting SIFT Features
Extracting SIFT Features
Extracting SIFT Features
Best preprocessing configuration for SIFT: {'filter': 'bilateral', 'filter_params': (9, 75, 75), 'histogram': 'clahe', 'normalize': False, 'f1-score': 0.804676279657609}
Extracting ORB Features
Extracting ORB Features
Extracting ORB Features
Best preprocessing configuration for ORB: {'filter': 'bilateral', 'filter_params': (9, 75, 75), 'histogram': 'none', 'normalize': False, 'f1-score': 0.6053571428571429}
Extracting Harris Corner Features
Extracting Harris Corner Features
Extracting Harris Corner Features
Best preprocessing configuration for Harris: {'filter': 'bilateral', 'filter_params': (5, 50, 50), 'histogram': 'clahe', 'normalize': True, 'f1-score': 0.7614035087719297}
Extracting Edge features
Extracting Edge features
Extracting Edge features
Best preprocessing configuration for EDGE: {'filter': 'bilateral', 'filter_params': (9, 75, 75), 'histogram': 'standard', 'normalize': False, 'f1-score': 0.831074977

In [8]:
with open("../best_configs_processing.json", "w") as f:
    json.dump(best_configs, f, indent=4)